In [1]:
%run "建立深度学习数据集 tensor.ipynb"

data.shape (4111, 5)
test_x.shape (4000, 96, 5)
test_y.shape (4000, 16, 1)
(4000, 96, 5) (4000, 16, 1)
allnumber 4000
trainnum 3200
testnum 800

torch.Size([128, 96, 5])
torch.Size([128, 16, 1])
----------------
torch.Size([128, 96, 5])
torch.Size([128, 16, 1])
y_true (4015, 1)
y_true_train (3215, 1)
y_true_test (815, 1)


In [2]:
from torch import nn

In [3]:
class ANN(nn.Module):
    def __init__(self, feature_x=5, feature_y=1, hidden=64, num_layers=3, time_window=96, step=16):#hidden一般2/4/16的倍数
        super(ANN, self).__init__()
        self.feature_y = feature_y
        self.step = step
        
        flattened_input_dim = feature_x * time_window
        
        self.fc1 = nn.Linear(flattened_input_dim, hidden)
        self.relu1 = nn.ReLU()
        
        #
        self.fc2 = nn.Linear(hidden, hidden)
        self.tanh2 = nn.Tanh()
         
        self.fc3 = nn.Linear(hidden, hidden)
        self.elu3 = nn.ELU()

        self.fc_time = nn.Linear(hidden, step * feature_y) # 64，16
     
    def forward(self, x):
        x = x.view(x.size(0), -1)
        
        out = self.fc1(x)
        out = self.relu1(out)
         
        out = self.fc2(out)
        out = self.tanh2(out)
         
        out = self.fc3(out)
        out = self.elu3(out)

        out = self.fc_time(out)

        out = out.view(-1, self.step, self.feature_y)
        return out


In [4]:
class RNN(nn.Module):
    def __init__(self, feature_x=5, feature_y=1, hidden=64, num_layers=3, time_window=96, step=16):
        super(RNN, self).__init__()
        self.feature_y = feature_y
        self.step = step
        self.hidden = hidden
        self.num_layers = num_layers
        
        # Calculate the flattened input dimension
        flattened_input_dim = feature_x
        
        # RNN layer
        self.rnn = nn.RNN(input_size=flattened_input_dim, 
                          hidden_size=hidden, 
                          num_layers=num_layers, 
                          batch_first=True)
        
        # Mapping to the time steps
        self.fc_time = nn.Linear(hidden, step * feature_y)
     
    def forward(self, x):
        x = x.view(x.size(0), x.size(1), -1)

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden).to(x.device)
        
  
        out, _ = self.rnn(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = self.fc_time(out[:, -1, :])  # Taking the last time step's output
        out = out.view(-1, self.step, self.feature_y)
        return out



In [5]:
class CNN(nn.Module):
    def __init__(self, feature=5, feature_y=1, hidden=64, num_layers=5, time_window=96, step=16):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=feature, out_channels=hidden, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=hidden, out_channels=hidden, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=hidden, out_channels=hidden, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv1d(in_channels=hidden, out_channels=hidden, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv1d(in_channels=hidden, out_channels=hidden, kernel_size=3, stride=1, padding=1)
        
        conv_out_size = hidden * time_window
        
        # Readout layer
        self.fc1 = nn.Linear(conv_out_size, hidden)
        self.fc2 = nn.Linear(hidden, step * feature_y)
     
        self.step = step
        self.feature_y = feature_y
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        
        x = x.view(x.size(0), -1)
        
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = x.view(-1, self.step, self.feature_y)
        return x


In [7]:
class dx_GRU(nn.Module):
    def __init__(self, feature=5, feature_y=1, hidden=64,num_layers=3, time_window=96, step=16):
        super(dx_GRU, self).__init__()
        
        # self.fc0 = nn.Linear(feature, 16)
        self.gru = nn.GRU(feature, hidden, num_layers, batch_first=True, dropout=0.3)
        # self.gru = nn.GRU(feature, hidden, num_layers, batch_first=True, dropout=0.3, bidirectional=True)
        
        self.fc1 = nn.Linear(time_window, step)
        self.fc2 = nn.Linear(hidden, feature_y)
        #self.fc2 = nn.Linear(hidden*2, feature_y)

    def forward(self, x):
        # x = self.fc0(x)
        
        y, hidden  = self.gru(x)

        y = self.fc1(y.permute(0, 2, 1))
        y = self.fc2(y.permute(0, 2, 1))
        return y




In [8]:
class GRU(nn.Module):
    def __init__(self, feature=5, feature_y=1, hidden=64,num_layers=3, time_window=96, step=16):
        super(GRU, self).__init__()
        
        # self.fc0 = nn.Linear(feature, 16)
        #self.gru = nn.GRU(feature, hidden, num_layers, batch_first=True, dropout=0.3)
        self.gru = nn.GRU(feature, hidden, num_layers, batch_first=True, dropout=0.3, bidirectional=True)
        
        self.fc1 = nn.Linear(time_window, step)
        #self.fc2 = nn.Linear(hidden, feature_y)
        self.fc2 = nn.Linear(hidden*2, feature_y)

    def forward(self, x):
        # x = self.fc0(x)
        
        y, hidden  = self.gru(x)

        y = self.fc1(y.permute(0, 2, 1))
        y = self.fc2(y.permute(0, 2, 1))
        return y


In [9]:
class LSTM(nn.Module):
    def __init__(self, feature=5, feature_y=1, hidden=64,num_layers=3, time_window=96, step=16,kernel_size=1,stride=1):
        super(LSTM, self).__init__()
        
        # self.fc0 = nn.Linear(feature, 16)
        # self.transformer = nn.TransformerEncoder(encoder_layer=nn.TransformerEncoderLayer(d_model=16, nhead=8), num_layers=3)
        #64*96*5
    
        self.lstm = nn.LSTM(5, hidden, num_layers, batch_first=True, dropout=0.3)
        
        self.fc1 = nn.Linear(time_window, step)
        # self.fc2 = nn.Linear(hidden*2, feature_y)
        self.fc2 = nn.Linear(hidden, feature_y)
        
    def forward(self, x):
        # x = self.fc0(x)
        # x = self.transformer(x)
        # y, (hidden, context) = self.lstm(x)
        
        y, hidden  = self.lstm(x)
        
        
        # y = self.cnn(y.permute(0, 2, 1))
        # y = self.fc1(y)
        # y = self.fc2(y.permute(0, 2, 1))

        y = self.fc1(y.permute(0, 2, 1))
        y = self.fc2(y.permute(0, 2, 1))
        return y

In [10]:
class BiLSTM(nn.Module):
    def __init__(self, feature=5, feature_y=1, hidden=64,num_layers=3, time_window=96, step=16,kernel_size=1,stride=1):
        super(BiLSTM, self).__init__()
        
        # self.fc0 = nn.Linear(feature, 16)
        # self.transformer = nn.TransformerEncoder(encoder_layer=nn.TransformerEncoderLayer(d_model=16, nhead=8), num_layers=3)
        self.lstm = nn.LSTM(5, hidden, num_layers, batch_first=True, dropout=0.3, bidirectional=True)
        self.fc1 = nn.Linear(time_window, step)
        # self.fc2 = nn.Linear(hidden*2, feature_y)
        self.fc2 = nn.Linear(hidden*2, feature_y)
        
    def forward(self, x):
        # x = self.fc0(x)
        # x = self.transformer(x)
        # y, (hidden, context) = self.lstm(x)
        
        y, hidden  = self.lstm(x)
        
        
        # y = self.cnn(y.permute(0, 2, 1))
        # y = self.fc1(y)
        # y = self.fc2(y.permute(0, 2, 1))

        y = self.fc1(y.permute(0, 2, 1))
        y = self.fc2(y.permute(0, 2, 1))
        return y

In [3]:
class CNN_LSTM(nn.Module):

    def __init__(self, input_size=5, hidden_size=64, num_layers=3, seq_length=96, output_size=16, kernel_size=3, stride=1):
        super(CNN_LSTM, self).__init__()
        
        self.conv = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=kernel_size, stride=stride, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        conv_out_size = (seq_length - kernel_size + 2 * 1) // stride + 1
        pool_out_size = conv_out_size // 2
        
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, output_size * 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = x.permute(0, 2, 1)
        
        out, _ = self.lstm(x)
        
        out = self.fc(out[:, -1, :])
        out = out.view(out.size(0), -1, 1)
        return out

In [4]:
class TransformerLSTM(nn.Module):
    def __init__(self, feature_x=5, feature_y=1, hidden=64, num_layers=3, time_window=96, step=16, kernel_size=1, stride=1):
        super(TransformerLSTM, self).__init__()
        
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=feature_x,
            nhead=5, 
            dim_feedforward=2048, 
            dropout=0.3
        )
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=1)
        
        self.lstm = nn.LSTM(input_size=feature_x, hidden_size=hidden, num_layers=6, batch_first=True)
        
        self.fc1 = nn.Linear(time_window, step)
        self.fc2 = nn.Linear(hidden, feature_y)
        

    def forward(self, x):
        # Transformer forward
        transformer_out = self.transformer(x)
        
        # LSTM forward
        lstm_out, _ = self.lstm(transformer_out)

        #batch_size, seq_len, d_model = transformer_out.size()
        #out = transformer_out.view(batch_size * seq_len, d_model)
        y = self.fc1(lstm_out.permute(0, 2, 1))
        y = self.fc2(y.permute(0, 2, 1))

        return y


In [11]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.input_attention = InputAttention(input_size, hidden_size)
        self.lstm = nn.LSTMCell(input_size, hidden_size)

    def forward(self, x):
        h = torch.zeros(x.size(0), self.lstm.hidden_size).to(device)
        c = torch.zeros(x.size(0), self.lstm.hidden_size).to(device)

        enc_outputs = []

        for t in range(x.size(1)):
            input_weights = self.input_attention(x[:, -time_window+t:, :], h)
            weighted_input = torch.mul(input_weights, x[:, -time_window+t:, :])
            h, c = self.lstm(weighted_input.mean(dim=1), (h, c))
            enc_outputs.append(h.unsqueeze(1))

        enc_outputs = torch.cat(enc_outputs, dim=1)
        return enc_outputs[:, -16:, :]

class InputAttention(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(InputAttention, self).__init__()
        self.W1 = nn.Linear(input_size, hidden_size)
        self.W2 = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1)

    def forward(self, x, h):
        h = h.repeat(x.size(1), 1, 1).transpose(0, 1)
        x = self.W1(x)
        h = self.W2(h)
        e = torch.tanh(x + h)
        attention_weights = torch.softmax(self.v(e), dim=1)
        return attention_weights


class TemporalAttention(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TemporalAttention, self).__init__()
        self.U = nn.Linear(input_size, hidden_size)
        self.W = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1)

    def forward(self, x, h):
        h = h.repeat(x.size(1), 1, 1).transpose(0, 1)
        x = self.U(x)
        h = self.W(h)
        e = torch.tanh(x + h)
        attention_weights = torch.softmax(self.v(e), dim=1)
        return attention_weights

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.temporal_attention = TemporalAttention(hidden_size, hidden_size)
        self.lstm = nn.LSTMCell(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, enc_outputs):
        h = torch.zeros(enc_outputs.size(0), self.lstm.hidden_size).to(enc_outputs.device)
        c = torch.zeros(enc_outputs.size(0), self.lstm.hidden_size).to(enc_outputs.device)

        dec_outputs = []
        for t in range(enc_outputs.size(1)):
            temporal_weights = self.temporal_attention(enc_outputs, h)
            context_vector = torch.sum(temporal_weights * enc_outputs, dim=1)
            h, c = self.lstm(context_vector, (h, c))
            out = self.fc(h)
            dec_outputs.append(out.unsqueeze(1))

        dec_outputs = torch.cat(dec_outputs, dim=1)
        return dec_outputs

class DA_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DA_RNN, self).__init__()
        self.encoder = Encoder(input_size,hidden_size)
        self.decoder = Decoder(hidden_size, output_size)

    def forward(self, x):
       # print(x.shape)
        enc_outputs = self.encoder(x)
        #print(enc_outputs.shape)   
        dec_output = self.decoder(enc_outputs)
        return dec_output

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

# Defines the overall TCN structure
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class TCNModel(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCNModel, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout)
        self.adaptive_pool = nn.AdaptiveAvgPool1d(output_size)
        self.linear = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.tcn(x)
        x = self.adaptive_pool(x)
        x = x.transpose(1, 2)
        x = self.linear(x)
        return x

In [7]:
class TrendSeasonalDecomposition(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.trend = nn.Linear(in_features, in_features)
        self.seasonal = nn.Linear(in_features, in_features)

    def forward(self, x):
        trend_part = self.trend(x)
        seasonal_part = self.seasonal(x)
        return trend_part, seasonal_part

class EnhancedMultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super().__init__()
        self.heads = heads
        self.embed_size = embed_size

        self.values = nn.Linear(embed_size, embed_size * heads)
        self.keys = nn.Linear(embed_size, embed_size * heads)
        self.queries = nn.Linear(embed_size, embed_size * heads)
        self.fc_out = nn.Linear(heads * embed_size, embed_size)

    def forward(self, values, keys, queries, mask):
        B = queries.shape[0]
        values = self.values(values).view(B, -1, self.heads, self.embed_size).transpose(1,2)
        keys = self.keys(keys).view(B, -1, self.heads, self.embed_size).transpose(1,2)
        queries = self.queries(queries).view(B, -1, self.heads, self.embed_size).transpose(1,2)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float('-inf'))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(B, -1, self.heads * self.embed_size)
        
        return self.fc_out(out)
    
    
class AutoformerEncoder(nn.Module):
    def __init__(self, feature_size, heads, num_layers, dropout):
        super().__init__()
        self.layer_stack = nn.ModuleList([
            nn.ModuleList([
                TrendSeasonalDecomposition(feature_size),
                EnhancedMultiHeadAttention(feature_size, heads),
                nn.LayerNorm(feature_size),
                nn.Dropout(dropout)
            ]) for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        for layer in self.layer_stack:
            trend_layer, attention_layer, norm_layer, dropout_layer = layer
            trend, seasonal = trend_layer(x)
            x = attention_layer(seasonal, seasonal, seasonal, mask) + trend
            x = norm_layer(x)
            x = dropout_layer(x)
        return x

In [10]:
class AutoformerTCN(nn.Module):
    def __init__(self, feature_size, num_channels, kernel_size, dropout, heads, num_layers):
        super().__init__()
        self.encoder = AutoformerEncoder(feature_size, heads, num_layers, dropout)
        self.decoder = TemporalConvNet(feature_size, num_channels, kernel_size, dropout)
        self.adaptive_pool = nn.AdaptiveAvgPool1d(16)
        
        self.final_layer = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        x = self.encoder(x)
        x = x.transpose(1, 2)
        x = self.decoder(x)
        x = self.adaptive_pool(x)
        
        x = self.final_layer(x.transpose(1, 2))
        return x